# Tutorial 1: Standard REXEE simulations

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/wehs7661/ensemble_md/c0f6d48ce3fe746e349e4a4a9610f935cca8b0b5?urlpath=lab%2Ftree%2Fdocs%2Fexamples%2Ftutorial_1%2Frun_REXEE.ipynb)

In this tutorial, we will demonstrate how one can use the different command-line interfaces (CLIs) implemented in the Python package `ensemble_md` to prepare, perform, and analyze a REXEE simulation to estimate the solvation free energy of a toy molecule composed of 4 interaction sites. For a more comprehensive understanding, we strongly recommend reading our [documentation](https://ensemble-md.readthedocs.io/en/latest/simulations.html) on launching REXEE simulations before starting this tutorial. With MPI (`mpirun` or `mpiexec`), GROMACS and `ensemble_md` all installed, you should be able to run this tutorial either locally or on a HPC cluster, as long as you have at least four CPU cores. Alternatively, you can click the badge above to run this tutorial on Binder without installing anything. Notably, this tutorial assumes that you understand the basics of the expanded ensemble (EE) method and relevant simulation parameters in GROMACS. If not, we recommend reading the [GROMACS documentation](https://manual.gromacs.org/current/reference-manual/algorithms/expanded-ensemble.html) and/or [this tutorial](https://weitsehsu.com/course/advanced_sampling/exe/). 

## 0. Background knowledge: A two-stage protocol

In methods such as expanded ensembles or REXEE, where each alchemical state has an associated weight (see [here](https://ensemble-md.readthedocs.io/en/latest/simulations.html)), it is common to use a two-stage protocol for alchemical free energy calculations. Specifically, this protocol is composed of the following two stages:

- A **weight-updating stage**: In a weight-updating stage, the weights of the different states are constantly updated until convergence during the simulation, usually by methods such as the Wang-Landau algorithm or its $1/t$ variant. The purpose of this stage is to obtain weights that could flatten out the free energy surface along the alchemical direction such that all states have a roughly equal chance to be visited in the next stage, i.e., the fixed-weight stage. 
- A **fixed-weight stage**: In a fixed-weight stage, the weights of different alchemical states are fixed at predefined values usually converged in a weight-updating stage. Unlike weight-updating simulations, a fixed-weight simulation is in equilibrium, and can therefore be analyzed by free energy estimators that require equilibrium data for free energy calculations, such as TI, BAR, MBAR, etc. 

With this protocol, one can use a weight-updating EE/REXEE simulation followed by a fixed-weight EE/REXEE simulation. In our [paper](https://pubs.acs.org/doi/10.1021/acs.jctc.4c00484), we have compared EE and REXEE methods in both weight-updating and fixed-weight phases comprehensively. 

Notably, this tutorial will only focus on fixed-weight REXEE simulations, with the weights provided by a weight-updating EE simulation not shown here. A weight-updating REXEE simulation can be performed simply by using the same workflow for running fixed-weight REXEE simulations, with input MDP files replaced by the ones for weight-updating EE simulations.

## 1. Preparing simulation inputs for a REXEE simulation

As mentioned in our documentation, running a REXEE simulation at least requires the following four input files:

- One YAML file that specifies REXEE parameters.
- One GROMACS GRO file of the system of interst.
- One GROMACS TOP file of the system of interst.
- One GROMACS MDP template for customizing MDP files for different replicas during the simulation.

In the folder where this tutorial resides (`docs/examples/tutoria_1` in the repository), you should find all these necessary input files, including `params.yaml`, `sys.gro`, `sys.top`, and `expanded.mdp`.

In [1]:
!ls

expanded.mdp  run_REXEE_log.txt  sim_1	sim_3		      sys.gro
params.yaml   sim_0		 sim_2	standard_REXEE.ipynb  sys.top


While you could use multiple GRO and TOP files, one for each replica in a REXEE simulation, in this tutorial, we will use the same GRO and TOP files for all replicas to keep the demonstration straightforward. As for the MDP template `expanded.mdp`, it can be inspected that it adopts common/typical settings for a fixed-weight EE simulation, since here our goal is to set up a fixed-weight REXEE simulation. In the MDP template, it can also be seen that 9 alchemical intermediate states are defined to decouple van der Waals interactions and coulombic interactions. 

During the REXEE simulation, customized MDP files will be generated for different replicas such that the MDP file for each replica only consider the set of states that the replica is constrained to. Notably, the weights specified via the parameter `init_lambda_weights` were obtained from a weight-updating EE simulation. If you would like to run a weight-updating REXEE simulation, simply use an MDP file for running a weight-updating EE simulation, i.e., and MDP file without `init_lambda_weights` being specified, and with options such as `wl_scale`, `wl_ratio`, `init_wl_delta`, `lmc_stats`, `lmc_weights_equil`, `weight_equil_wl_delta` specified.

Now, with these three GROMACS inputs prepared, let's briefly review the input YAML file that specifies REXEE parameters.

In [2]:
!cat params.yaml

# Simulation parameters
gmx_executable: 'gmx'               
gro: sys.gro
top: sys.top
mdp: expanded.mdp             
n_sim: 4
n_iter: 5         
s: 1                         
nst_sim: 500
runtime_args: {'-nt': '1'}

# Data analysis parameters
free_energy: True


As a minimum working example, this YAML file only specifies the compulsory simulation parameters and adopts default values for all optional parameters, execpt that `runtime_args` is specified to run each EE replica using one thread (`-nt 1`) so that it would not take too long to complete the simulation. For information about all possible parameters that can be specified in the YAML file, please refer to our [documentation](https://ensemble-md.readthedocs.io/en/latest/simulations.html#input-yaml-parameters).

In our case, we will run our a REXEE simulation composed of 4 replicas, with each replica performed with one thread (`-nt 1`). With the total number of states as 9 and a state shift of 1, this means that replicas 0, 1, 2, 3 are constrained to sampling states 0-5, 1-6, 2-7, and 3-8, respectively. The simulation will attempt to swap coordinates between replicas every 500 integration steps, and perform 5 iterations in total. (Note that generally we need a much larger number of iterations. Here we use a very small value just for demosntration purposes.) 

If you are interested in trying out other numbers of replicas or state shifts given 9 alchemical intermediate states, you can use the CLI `explore_REXEE` to enumerate all possible REXEE configurations.

In [3]:
!explore_REXEE -N 9

Exploration of the REXEE parameter space
[ REXEE parameters of interest ]
- N: The total number of states
- r: The number of replicas
- n: The number of states for each replica
- s: The state shift between adjacent replicas

[ Solutions ]
- Solution 1: (N, r, n, s) = (9, 2, 5, 4)
  - Replica 0: [0, 1, 2, 3, 4]
  - Replica 1: [4, 5, 6, 7, 8]

- Solution 2: (N, r, n, s) = (9, 2, 6, 3)
  - Replica 0: [0, 1, 2, 3, 4, 5]
  - Replica 1: [3, 4, 5, 6, 7, 8]

- Solution 3: (N, r, n, s) = (9, 2, 7, 2)
  - Replica 0: [0, 1, 2, 3, 4, 5, 6]
  - Replica 1: [2, 3, 4, 5, 6, 7, 8]

- Solution 4: (N, r, n, s) = (9, 2, 8, 1)
  - Replica 0: [0, 1, 2, 3, 4, 5, 6, 7]
  - Replica 1: [1, 2, 3, 4, 5, 6, 7, 8]

- Solution 5: (N, r, n, s) = (9, 3, 5, 2)
  - Replica 0: [0, 1, 2, 3, 4]
  - Replica 1: [2, 3, 4, 5, 6]
  - Replica 2: [4, 5, 6, 7, 8]

- Solution 6: (N, r, n, s) = (9, 3, 7, 1)
  - Replica 0: [0, 1, 2, 3, 4, 5, 6]
  - Replica 1: [1, 2, 3, 4, 5, 6, 7]
  - Replica 2: [2, 3, 4, 5, 6, 7, 8]

- Solution 7: (

If you want to list all possible configurations that use 4 replicas for sampling 9 alchemical states, you can further specify the number of replicas via the flag `-r`:

In [4]:
!explore_REXEE -N 9 -r 4

Exploration of the REXEE parameter space
[ REXEE parameters of interest ]
- N: The total number of states
- r: The number of replicas
- n: The number of states for each replica
- s: The state shift between adjacent replicas

[ Solutions ]
- Solution 1: (N, r, n, s) = (9, 4, 3, 2)
  - Replica 0: [0, 1, 2]
  - Replica 1: [2, 3, 4]
  - Replica 2: [4, 5, 6]
  - Replica 3: [6, 7, 8]

- Solution 2: (N, r, n, s) = (9, 4, 6, 1)
  - Replica 0: [0, 1, 2, 3, 4, 5]
  - Replica 1: [1, 2, 3, 4, 5, 6]
  - Replica 2: [2, 3, 4, 5, 6, 7]
  - Replica 3: [3, 4, 5, 6, 7, 8]



For rules of thumb about specifying the configurational parameters (including the number of states, number of replicas, state shift and swapping frequency), please refer to our [documentation](https://ensemble-md.readthedocs.io/en/latest/simulations.html#some-rules-of-thumb) or [paper](https://pubs.acs.org/doi/epdf/10.1021/acs.jctc.4c00484).

## 2. Performing a REXEE simulation
With all the input files in place, we can now run a fixed-weight REXEE simulation using the CLI `run_REXEE` with the command shown below. 

If you are using a laptop with 4 CPU cores, this should take around 15 minutes to complete. If you run the tutorial on Binder, this should take around XXX minutes. Note that here we use default values for all flags of the CLI `run_REXEE`. For more information about the flags, please run `run_REXEE -h`/`run_REXEE --help` and read the help message.

In [8]:
!mpirun -np 4 run_REXEE

Current time: 29/07/2024 18:51:10
Command line: /srv/conda/envs/notebook/bin/run_REXEE

Important parameters of REXEE
Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
GROMACS executable: /srv/conda/envs/notebook/bin.AVX2_256/gmx
GROMACS version: 2023.3-conda_forge
ensemble_md version: 0.9.0+159.g813e06f.dirty
Simulation inputs: sys.gro, sys.top, expanded.mdp
Verbose log file: True
Proposal scheme: exhaustive
Whether to perform weight combination: False
Type of means for weight combination: simple
Whether to perform histogram correction: False
Histogram cutoff for weight correction: -1
Number of replicas: 4
Number of iterations: 5
Length of each replica: 1.0 ps
Frequency for checkpointing: 100 iterations
Total number of states: 9
Additionally defined swappable states: None
Additional grompp arguments: None
Additional runtime arguments: {'-nt': '1'}
External modules for coordinate manipulation: None
MDP parameters differing across replicas: N

As shown above, the CLI `run_REXEE` printed information about the simulation on-the-fly, including the number of iterations completed, information about the current iteration, whether a swap was accepted or not, etc. At the end, a summary of the simulation was printed. All such information is saved in `run_REXEE_log.txt`. Below we list the files in the current directory and print the directory structure to see what outputs have been generated by the simulation:

In [9]:
!ls

expanded.mdp  rep_trajs.npy	 sim_0	sim_2  standard_REXEE.ipynb  sys.top
params.yaml   run_REXEE_log.txt  sim_1	sim_3  sys.gro


In [10]:
!tree sim_0

sim_0
├── iteration_0
│   ├── confout.gro
│   ├── dhdl.xvg
│   ├── ener.edr
│   ├── expanded.mdp
│   ├── md.log
│   ├── mdout.mdp
│   ├── sys_EE.tpr
│   └── traj_comp.xtc
├── iteration_1
│   ├── confout.gro
│   ├── dhdl.xvg
│   ├── ener.edr
│   ├── expanded.mdp
│   ├── md.log
│   ├── mdout.mdp
│   ├── sys_EE.tpr
│   └── traj_comp.xtc
├── iteration_2
│   ├── confout.gro
│   ├── dhdl.xvg
│   ├── ener.edr
│   ├── expanded.mdp
│   ├── md.log
│   ├── mdout.mdp
│   ├── sys_EE.tpr
│   └── traj_comp.xtc
├── iteration_3
│   ├── confout.gro
│   ├── dhdl.xvg
│   ├── ener.edr
│   ├── expanded.mdp
│   ├── md.log
│   ├── mdout.mdp
│   ├── sys_EE.tpr
│   └── traj_comp.xtc
└── iteration_4
    ├── confout.gro
    ├── dhdl.xvg
    ├── ener.edr
    ├── expanded.mdp
    ├── md.log
    ├── mdout.mdp
    ├── sys_EE.tpr
    └── traj_comp.xtc

6 directories, 40 files


As shown above, the following files/directories have been generated:
- Folders `sim_*`, each of which corresponds to a replica in the REXEE simulation and contains subfolders `iteration_*`, where simulation outputs of each iteration reside.
- `rep_trajs.npy`, which contains replica-space trajectories for all replicas and is used as an input checkpoint file when simulation extension is desired. To extend a REXEE simulation, one only needs to input `rep_trajs.npy` via the `-c` flag of `run_REXEE`.
- `run_REXEE_log.txt`, a log file as mentioned above.

Notably, if a weight-updating REXEE simulation is performed, `g_vecs.npy` will be generated, which contains the times series of alchemical weights for all replicas.

## 3. Analyzing a REXEE simulation

After the simulation is completed, we are ready to analyze the simulation to assess its sampling behaviors and estimate the solvation free energy of the molecule. Since the simulation we performed in the previous section is very short and was just for demonstration purposes, instead of using the simulation outputs above, we will unzip `example_outputs.zip` and analyze the example outputs, using the CLI `analyze_REXEE`.

To set up an example for analyzing a REXEE simulation, we first delete the simulation outputs generated by the previous section, and unzip `example_outputs.zip`.

In [ ]:
# remove the files and unzip example_output.zip here.

Importantly, the CLI `analyze_REXEE` also requires an input YAML file, which can be the same as the one used for `run_REXEE`, since these two CLIs only consider their own relevant parameters. Regardless of the settings in the input YAML file, the CLI `analyze_REXEE` will perform the following analyses. (For more information, please check the [API documentation](https://ensemble-md.readthedocs.io/en/latest/api/api_analysis.html) of the relevant functionalities, or the [source code](https://github.com/wehs7661/ensemble_md/blob/master/ensemble_md/cli/analyze_REXEE.py) of the CLI `analyze_REXEE`.)

- Read in and plot replica-space trajectories (using `ensemble_md.analyze_traj.plot_rep_trajs`)
- Calculate and plot the replica-space transition matrix (using `ensemble_md.analyze_traj.tarj2transmtx` and `ensemble_md.analyze_matrix.plot_matrix`)
- Calculate the spectral gap for the replica-space transition matrix (using `ensemble_md.analyze_matrix.calc_spectral_gap`) 
- Recover all configurationally continuous trajectories in the state space (using `ensemble_md.analyze_traj.stitch_time_series`)
- Plot the state-space time series for different continuous trajectories (using `ensemble_md.analyze_traj.plot_state_trajs`)
- Plot the histograms of state index for different continuous trajectories (using `ensemble_md.analyze_traj.plot_state_hist`)
- Stitch the time series of state index for different sets of states (using `ensemble_md.analyze_traj.stitch_time_series_for_sim`)
- Plot the time series of state index for different sets of states (using `ensemble_md.analyze_traj.plot_state_trajs`)
- Plot the histograms of state index for different sets of states (using `ensemble_md.analyze_traj.plot_state_hist`)
- Plot the overall state-space transition matrices for different continuous trajectories (using `ensemble_md.analyze_traj.traj2transmtx` and `ensemble_md.analyze_matrix.plot_matrix`)
- For each continuous trajectory, calculate the state-space spectral gap (using `ensemble_md.analyze_matrix.calc_spectral_gap`)
- For each continuous trajectory, calculate the state-space stationary distribution (using `ensemble_md.analyze_matrix.calc_equil_prob`)
- Calculate the state index correlation time for each continuous trajectory (using `pymbar.timeries.statistical_inefficiency`)
- Calculate transit times for each continuous trajectory (using `ensemble_md.analyze_traj.plot_transit_time`)

Additionally, MSM analysis and free energy calculations can be performed if parameters `msm` and `free_energy` are set to `True`, respectively. 


As shown above, we did set `free_energy` to `True` in `params.yaml` (with all other data analysis parameters adopting their default values), so free energy calculations will be performed when the CLI `analyze_REXEE` is used. 

Now, to perform data analysis for the example outputs, simply execute the command below. Note that the CLI `analyze_REXEE` does not support parallelism and here we are using default values for all the flags. For more information about the flags, please run `analyze_REXEE -h`/`analyze_REXEE --help` and read the help message.

In [11]:
!mpirun -np 1 analyze_REXEE

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************
Current time: 29/07/2024 18:54:28
Command line: /srv/conda/envs/notebook/bin/analyze_REXEE
Important parameters of REXEE
Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
GROMACS executable: /srv/conda/envs/notebook/bin.AVX2_256/gmx
GROMACS version: 2023.3-conda_forge
ensemble_md version: 0.9.0+159.g813e

As shown above, all data analysis results were not only printed on the fly to the screen, but also printed to the file `analyze_REXEE_log.txt`. 

Now, as the execution of the command `analyze_REXEE` has been completed, let's take a look and the data analysis results saved in the newly created folder `analysis`. 

In [12]:
!ls analysis

rep_trajs.png  rep_transmtx_allconfigs.png
